In [1]:
import os
os.chdir('../../..')

This notebook demonstrates the use of vectors in ConvoKit, as well as the use of the bag-of-words transformer, BoWTransformer, and the 

In [2]:
import convokit

In [3]:
from convokit import Corpus, download

In [4]:
corpus = Corpus(download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [5]:
corpus.print_summary_stats()

Number of Speakers: 7568
Number of Utterances: 74467
Number of Conversations: 10744


## Using a Transformer to add a vector

In [6]:
from convokit import VectorClassifier, BoWTransformer

In [7]:
bow_transformer = BoWTransformer(obj_type="utterance", vector_name='bow')

Initializing default unigram CountVectorizer...Done.


In [8]:
random_utt = corpus.random_utterance()
random_utt

Utterance({'obj_type': 'utterance', '_owner': <convokit.model.corpus.Corpus object at 0x12cfa28d0>, 'meta': {'score': 0, 'top_level_comment': None, 'retrieved_on': 1441167958, 'gilded': 0, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/2gl6jw/tips_for_transfer_applicant/', 'author_flair_text': ''}, '_id': '2gl6jw', 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', '_owner': <convokit.model.corpus.Corpus object at 0x12cfa28d0>, 'meta': {'num_posts': 2194, 'num_comments': 6009}, '_id': '[deleted]', 'vectors': []}), 'conversation_id': '2gl6jw', '_root': '2gl6jw', 'reply_to': None, 'timestamp': 1410895653, 'text': "Hi /r/Cornell. I'm a CS student who is currently a freshman at a state school looking to transfer to Cornell next fall, one to get a better education, change of scenery, and want to be closer to some extended family in Ithaca. Anything I should know/classes I should take?"})

In [9]:
random_utt.vectors

[]

In [10]:
corpus.vectors

[]

In [11]:
bow_transformer.fit_transform(corpus)

In [12]:
random_utt

Utterance({'obj_type': 'utterance', '_owner': <convokit.model.corpus.Corpus object at 0x12cfa28d0>, 'meta': {'score': 0, 'top_level_comment': None, 'retrieved_on': 1441167958, 'gilded': 0, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/2gl6jw/tips_for_transfer_applicant/', 'author_flair_text': ''}, '_id': '2gl6jw', 'vectors': ['bow'], 'speaker': Speaker({'obj_type': 'speaker', '_owner': <convokit.model.corpus.Corpus object at 0x12cfa28d0>, 'meta': {'num_posts': 2194, 'num_comments': 6009}, '_id': '[deleted]', 'vectors': []}), 'conversation_id': '2gl6jw', '_root': '2gl6jw', 'reply_to': None, 'timestamp': 1410895653, 'text': "Hi /r/Cornell. I'm a CS student who is currently a freshman at a state school looking to transfer to Cornell next fall, one to get a better education, change of scenery, and want to be closer to some extended family in Ithaca. Anything I should know/classes I should take?"})

In [13]:
random_utt.vectors

['bow']

### Fetching the vector for the utterance

In [14]:
random_utt.get_vector('bow')

<1x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 37 stored elements in Compressed Sparse Row format>

In [15]:
random_utt.get_vector('bow', as_dataframe=True)

,00,000,00am,00pm,01,02,03,04,05,06,...,youtu,youtube,yr,yrs,yup,zero,zeus,zimride,zip,zone
2gl6jw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
random_utt.get_vector('bow', as_dataframe=True, columns=['youtu', 'youtube', 'yr'])

,youtu,youtube,yr
2gl6jw,0,0,0


In [17]:
random_utt.get_vector('bow', as_dataframe=False, columns=['youtu', 'youtube', 'yr'])

array([[0, 0, 0]])

### What does this look like at the Corpus level?

In [18]:
corpus.vectors

['bow']

In [19]:
corpus.get_vector_matrix('bow')

ConvoKitMatrix('name': bow, 'matrix': <74467x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 2108383 stored elements in Compressed Sparse Row format>)

In [20]:
bow_matrix = corpus.get_vector_matrix('bow')

In [21]:
bow_matrix.to_dataframe().head()

,00,000,00am,00pm,01,02,03,04,05,06,...,youtu,youtube,yr,yrs,yup,zero,zeus,zimride,zip,zone
nyx4d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
o0145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
o1gca,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
o0ss4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
o31u0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
bow_matrix.columns[:10]

['00', '000', '00am', '00pm', '01', '02', '03', '04', '05', '06']

In [23]:
bow_matrix.ids[:10]

['nyx4d',
 'o0145',
 'o1gca',
 'o0ss4',
 'o31u0',
 'o4ipd',
 'o456r',
 'o4544',
 'o3l7i',
 'o3fqm']

In [24]:
bow_matrix.name

'bow'

In [25]:
bow_matrix.matrix

<74467x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 2108383 stored elements in Compressed Sparse Row format>

## Dumping and loading

### Dumps all vectors by default

In [26]:
# dumps all vectors by default
corpus.dump('cornell-with-bow', base_path='convokit/classifier/demo')

In [27]:
os.listdir('convokit/classifier/demo/cornell-with-bow')

['utterances.jsonl',
 'conversations.json',
 'vectors.bow.p',
 'corpus.json',
 'speakers.json',
 'index.json']

### But vectors can be excluded

In [28]:
corpus.dump('cornell-no-bow', base_path='convokit/classifier/demo', exclude_vectors=['bow'])

In [29]:
os.listdir('convokit/classifier/demo/cornell-no-bow')

['utterances.jsonl',
 'conversations.json',
 'corpus.json',
 'speakers.json',
 'index.json']

### When the corpus is loaded, vectors are present 'structurally' but not actually loaded

In [36]:
corpus = Corpus(filename='convokit/classifier/demo/cornell-with-bow')

In [37]:
corpus.vectors

['bow']

In [38]:
corpus.random_utterance().vectors

[]

In [32]:
corpus._vector_matrices # private variable storing vector matrices

{}

In [33]:
# when fetched normally
corpus.get_vector_matrix('bow')

ConvoKitMatrix('name': bow, 'matrix': <74467x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 2108383 stored elements in Compressed Sparse Row format>)

In [34]:
corpus._vector_matrices

{'bow': ConvoKitMatrix('name': bow, 'matrix': <74467x9340 sparse matrix of type '<class 'numpy.int64'>'
 	with 2108383 stored elements in Compressed Sparse Row format>)}

### We can also load the corpus with vectors fully loaded

In [39]:
corpus = Corpus(filename='convokit/classifier/demo/cornell-with-bow', vectors=['bow'])

In [41]:
corpus._vector_matrices # private variable storing matrices

{'bow': ConvoKitMatrix('name': bow, 'matrix': <74467x9340 sparse matrix of type '<class 'numpy.int64'>'
 	with 2108383 stored elements in Compressed Sparse Row format>)}

In [ ]:
corpus.get_vec

In [35]:
input()

KeyboardInterrupt: 

### Training a VectorClassifier

Setting up a basic prediction task where we predict whether an Utterance (corresponding to a Reddit comment) has a score above 0 based on its bag-of-words vector.

In [ ]:
bow_classifier = VectorClassifier(obj_type="utterance", labeller=lambda utt: utt.meta['score'] > 0)

In [ ]:
bow_classifier.fit(corpus, vector_name='bow')

In [ ]:
bow_classifier.transform(corpus, vector_name='bow')

In [ ]:
bow_classifier.summarize(corpus).head()

In [ ]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).head()

In [ ]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).tail()

In [ ]:
y_true, y_pred = bow_classifier.get_y_true_pred(corpus)

In [ ]:
bow_classifier.base_accuracy(corpus)

In [ ]:
bow_classifier.accuracy(corpus)

In [ ]:
print(bow_classifier.classification_report(corpus))

## A bag-of-words prediction task at the Conversation-level

For each conversation, based on the bag-of-words of the first 5 utterances, we ask the question of whether the comment thread will eventually double in length or stay the same length. 

In [ ]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [ ]:
corpus.print_summary_stats()

In [ ]:
len(top_level_comment_ids)

In [ ]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids)

In [ ]:
threads_corpus.print_summary_stats()

In [ ]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [ ]:
from collections import Counter

In [ ]:
# Count of negative / positive examples
Counter(threads_corpus.get_conversations_dataframe()['meta.thread_doubles'])

In [ ]:
bow_transformer2 = BoWTransformer(obj_type="conversation", vector_name='first_5_BoW')

In [ ]:
bow_transformer2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
threads_corpus.vectors

In [ ]:
bow_classifier2 = VectorClassifier(obj_type="conversation", 
                    labeller=lambda convo: convo.meta['thread_doubles'])

In [ ]:
bow_classifier2.fit_transform(threads_corpus, vector_name='first_5_BoW',
                              selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
summary = bow_classifier2.summarize(threads_corpus, 
                                    selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
summary.head()

In [ ]:
summary.tail()

In [ ]:
bow_classifier2.base_accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
bow_classifier2.accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
print(bow_classifier2.classification_report(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None))

In [ ]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).head(10)

In [ ]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).tail(10)

In [ ]:
bow_classifier2.confusion_matrix(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
bow_classifier2.evaluate_with_cv(threads_corpus, vector_name='first_5_BoW', selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
bow_classifier2.evaluate_with_train_test_split(threads_corpus, vector_name='first_5_BoW',
                                               selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
objs = list(threads_corpus.iter_conversations(selector=lambda convo: convo.meta['thread_doubles'] is not None))

In [ ]:
objs = bow_classifier2.transform_objs(objs)

In [ ]:
# bow_classifier2.summarize_objs(objs)

# Here be dragons (i.e. to be updated)

## Paired bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [ ]:
from convokit import Pairer, PairedBoW

In [ ]:
corpus = Corpus(filename=download('subreddit-Cornell'))

In [ ]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids, preserve_convo_meta=True)

In [ ]:
next(threads_corpus.iter_conversations())

In [ ]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [ ]:
next(threads_corpus.iter_conversations()).meta

In [ ]:
pairer = Pairer(obj_type="conversation", 
                pos_label_func=lambda convo: convo.meta['thread_doubles'], 
                neg_label_func=lambda convo: convo.meta['thread_doubles'] == False,
                pairing_func=lambda convo: convo.meta['original_convo_id']
               )

In [ ]:
pairer.transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [ ]:
print(next(threads_corpus.iter_conversations()))

In [ ]:
for convo in threads_corpus.iter_conversations():
    if convo.meta['pair_id'] is not None:
        print(convo)
        break

In [ ]:
from convokit import BoWTransformer

In [ ]:
bow_transformer = BoWTransformer(obj_type="conversation", 
                                 text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                    )

In [ ]:
bow_transformer.fit_transform(threads_corpus, selector=lambda convo: convo.meta['pair_id'] is not None)

In [ ]:
paired_bow = PairedBoW(obj_type="conversation")

In [ ]:
paired_bow.fit(threads_corpus)

In [ ]:
paired_bow.summarize(threads_corpus)

In [ ]:
paired_bow.get_coefs(feature_names=bow_transformer.get_vocabulary())